### This script does two things, first it creates a dataset with information about each legistlator, next it creates a dataset of the relationships between each legislator ###

### This script should be run after the create_data script and before the analysis script

### Part 1: create legislator dataset ###

In [11]:
import pandas as pd
import numpy as np
from tokenize import generate_tokens
xrange = range
from io import StringIO

In [12]:
# Helper funtions to process an array of strings from a csv

def parts(a):
    """Split a python-tokenizable expression on comma operators"""
    compos = [-1] # compos stores the positions of the relevant commas in the argument string
    compos.extend(t[2][1] for t in generate_tokens(StringIO(a).readline) if t[1] == ',')
    compos.append(len(a))
    return [ a[compos[i]+1:compos[i+1]] for i in xrange(len(compos)-1)]

#coverts to list of strings
def convertType(toConv, makeUpper):
    
    tcs = []
    for tc in toConv:
        tc = tc.replace('[','')
        tc = tc.replace(']','')
        tc = tc.replace('{','')
        tc = tc.replace('}','')
        tc = tc.replace(' \'','')
        tc = tc.replace('\'','')
        tc = tc.replace('@','')
        tc = tc.replace('-','')
        tc = tc.replace('#','')
        chain = parts(tc)
        
        chain2 = []
        for c in chain:
            chain2.append(c) 
        
        #make uppercase if specified
        chain3 = []
        if makeUpper:
            for c in chain2:
                chain3.append(c.upper())
            tcs.append(chain3)
        else:
            tcs.append(chain2)
        
    return np.array(tcs)

In [25]:
# load dataset and fix all string fields 

df = pd.read_csv("bills_2019.csv")
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]

for column in df:
    if (type(df[column][0]) == str):
        df[column] = convertType(df[column], False)

print(len(df))

2497


In [33]:
print(len(df))
print(len(df.drop_duplicates('bill_number')))

2497
2497


In [14]:
## Here we find every legislator in our data, we create a dataset with some basic info about them

df = df.dropna()

leg_parties = []
leg_names = []
leg_districts = []
leg_roles = []
leg_people_ids = []
leg_committee_sponsors = []
for index, row in df.iterrows():
    names = row["sponsor_name"]
    parties = row["sponsor_party"]
    districts = row["sponsor_district"]
    roles = row["sponsor_role"]
    people_ids = row["sponsor_people_id"]
    committee_sponsors = row["committee_sponsor"]
    for name, party, district, role, people_id, committee_sponsor in zip(names,parties,districts,roles,people_ids,committee_sponsors):
        if (name.replace("\"","") not in leg_names and name != ""):
            leg_names.append(name.replace("\"",""))
            leg_roles.append(role)
            leg_people_ids.append(people_id)
            leg_committee_sponsors.append(committee_sponsor)
            if (party == ""):
                leg_parties.append("No Party")
            else:
                leg_parties.append(party)
            if (district == ""):
                leg_districts.append("No District")
            else:
                leg_districts.append(district)
            
legislators = pd.DataFrame({
    "name": leg_names,
    "party": leg_parties,
    "district" : leg_districts,
    "role" : leg_roles,
    "people_id" : leg_people_ids,
    "committee_sponsor" : leg_committee_sponsors
})

In [15]:
legislators.head()

,name,party,district,role,people_id,committee_sponsor
0,Stephen Lafferty,D,HD042A,Rep,4577,0
1,Cheryl Glenn,D,HD045,Rep,4544,0
2,Alfred Carr,D,HD018,Rep,4547,0
3,Mark Chang,D,HD032,Rep,17233,0
4,Eric Ebersole,D,HD012,Rep,17234,0


In [49]:
print(len(legislators))
print(len(df))

227
2497


In [55]:
## Here we count how many bills each legislator has sponsored, we also count each type of sponsorship
# and make a list of all of the bills and their status

# counts of bills
all_sponsors_count = []
primary_sponsors_count = []
co_sponsors_count = []
joint_sponsors_count = []
unspecified_sponsors_count = []

# lists of bills
all_bills = []
primary_bills = []
co_bills = []
joint_bills = []
unspecified_bills = []

# lists of bill status
all_bills_status = []
primary_bills_status = []
co_bills_status = []
joint_bills_status = []
unspecified_bills_status = []

bill_track = []

hits = 0
for leg_id in legislators["people_id"]:
    leg_id = leg_id.strip()
    l_all_bills = []
    l_primary_bills = []
    l_co_bills = []
    l_joint_bills = []
    l_unspecified_bills = []
    l_all_bills_status = []
    l_primary_bills_status = []
    l_co_bills_status = []
    l_joint_bills_status = []
    l_unspecified_bills_status = []
    for index, row in df.iterrows():
        hits = hits + 1
        sponsor_people_ids = row["sponsor_people_id"]
        sponsor_type_ids = row["sponsor_type_id"]
        this_person_track = []
        for people_id, type_id in zip(sponsor_people_ids,sponsor_type_ids):
            type_id = type_id.strip()
            people_id = people_id.strip()
            if (people_id == leg_id):
                l_all_bills.append(row["bill_id"])
                l_all_bills_status.append(row["status"])
                if (type_id == "0"):
                    l_unspecified_bills.append(row["bill_id"])
                    l_unspecified_bills_status.append(row["status"])
                elif (type_id == "1"):
                    if (row["bill_id"] not in bill_track):
                        bill_track.append(row["bill_id"])
                        l_primary_bills.append(row["bill_id"])
                        l_primary_bills_status.append(row["status"])
                elif (type_id == "2"):
                        l_co_bills.append(row["bill_id"])
                        l_co_bills_status.append(row["status"])
                elif (type_id == "3"):
                    l_joint_bills.append(row["bill_id"])
                    l_joint_bills_status.append(row["status"])
    
    all_bills.append(l_all_bills)
    all_bills_status.append(l_all_bills_status)
    unspecified_bills.append(l_unspecified_bills)
    unspecified_bills_status.append(l_unspecified_bills_status)
    primary_bills.append(l_primary_bills)
    primary_bills_status.append(l_primary_bills_status)
    co_bills.append(l_co_bills)
    co_bills_status.append(l_co_bills_status)
    joint_bills.append(l_joint_bills)
    joint_bills_status.append(l_joint_bills_status)
    
    all_sponsors_count.append(len(l_all_bills))
    unspecified_sponsors_count.append(len(l_unspecified_bills))
    primary_sponsors_count.append(len(l_primary_bills))
    co_sponsors_count.append(len(l_co_bills))
    joint_sponsors_count.append(len(l_joint_bills))
    

In [56]:
legislators["all_sponsored_bills_count"] = all_sponsors_count
legislators["unspecified_sponsored_bills_count"] = unspecified_sponsors_count
legislators["primary_sponsored_bills_count"] = primary_sponsors_count
legislators["co_sponsored_bills_count"] = co_sponsors_count
legislators["joint_sponsored_bills_count"] = joint_sponsors_count
legislators["all_sponsored_bills"] = all_bills
legislators["unspecified_sponsored_bills"] = unspecified_bills
legislators["primary_sponsored_bills"] = primary_bills
legislators["co_sponsored_bills"] = co_bills
legislators["joint_sponsored_bills"] = joint_bills
legislators["all_sponsored_bills_status"] = all_bills_status
legislators["unspecified_sponsored_bills_status"] = unspecified_bills_status
legislators["primary_sponsored_bills_status"] = primary_bills_status
legislators["co_sponsored_bills_status"] = co_bills_status
legislators["joint_sponsored_bills_status"] = joint_bills_status

all_sponsored_bills_passed = []
unspecified_sponsored_bills_passed = []
primary_sponsored_bills_passed = []
co_sponsored_bills_passed = []
joint_sponsored_bills_passed = []
for index, row in legislators.iterrows():
    all_sponsored_bills_passed.append(row["all_sponsored_bills_status"].count(4) + row["all_sponsored_bills_status"].count(3))
    unspecified_sponsored_bills_passed.append(row["unspecified_sponsored_bills_status"].count(4) + row["unspecified_sponsored_bills_status"].count(3))
    primary_sponsored_bills_passed.append(row["primary_sponsored_bills_status"].count(4) + row["primary_sponsored_bills_status"].count(3))
    co_sponsored_bills_passed.append(row["co_sponsored_bills_status"].count(4) + row["co_sponsored_bills_status"].count(3))
    joint_sponsored_bills_passed.append(row["joint_sponsored_bills_status"].count(4) + row["joint_sponsored_bills_status"].count(3))
 
legislators["all_sponsored_bills_passed"] = all_sponsored_bills_passed
legislators["unspecified_sponsored_bills_passed"] = unspecified_sponsored_bills_passed
legislators["primary_sponsored_bills_passed"] = primary_sponsored_bills_passed
legislators["co_sponsored_bills_passed"] = co_sponsored_bills_passed
legislators["joint_sponsored_bills_passed"] = joint_sponsored_bills_passed

In [57]:
# here we calculate avgs for each type of sponsorship. (what percent of bills were passed)

all_sponsored_bills_avg = []
unspecified_sponsored_bills_avg = []
primary_sponsored_bills_avg = []
co_sponsored_bills_avg = []
joint_sponsored_bills_avg = []
for index, row in legislators.iterrows():
    if (row["all_sponsored_bills_count"] == 0):
        all_sponsored_bills_avg.append(0.0)
    else:
        all_sponsored_bills_avg.append(row["all_sponsored_bills_passed"]/row["all_sponsored_bills_count"])
    if (row["unspecified_sponsored_bills_count"] == 0):
        unspecified_sponsored_bills_avg.append(0.0)
    else:
        unspecified_sponsored_bills_avg.append(row["unspecified_sponsored_bills_passed"]/row["unspecified_sponsored_bills_count"])
    if (row["primary_sponsored_bills_count"] == 0):
        primary_sponsored_bills_avg.append(0.0)
    else:
        primary_sponsored_bills_avg.append(row["primary_sponsored_bills_passed"]/row["primary_sponsored_bills_count"])
    if (row["co_sponsored_bills_count"] == 0):
        co_sponsored_bills_avg.append(0.0)
    else:
        co_sponsored_bills_avg.append(row["co_sponsored_bills_passed"]/row["co_sponsored_bills_count"])
    if (row["joint_sponsored_bills_count"] == 0):
        joint_sponsored_bills_avg.append(0.0)
    else:
        joint_sponsored_bills_avg.append(row["joint_sponsored_bills_passed"]/row["joint_sponsored_bills_count"])
 
legislators["all_sponsored_bills_avg"] = all_sponsored_bills_avg
legislators["unspecified_sponsored_bills_avg"] = unspecified_sponsored_bills_avg
legislators["primary_sponsored_bills_avg"] = primary_sponsored_bills_avg
legislators["co_sponsored_bills_avg"] = co_sponsored_bills_avg
legislators["joint_sponsored_bills_avg"] = joint_sponsored_bills_avg

In [58]:
legislators[100:200].head(50)

,name,party,district,role,people_id,committee_sponsor,all_sponsored_bills_count,unspecified_sponsored_bills_count,primary_sponsored_bills_count,co_sponsored_bills_count,...,all_sponsored_bills_passed,unspecified_sponsored_bills_passed,primary_sponsored_bills_passed,co_sponsored_bills_passed,joint_sponsored_bills_passed,all_sponsored_bills_avg,unspecified_sponsored_bills_avg,primary_sponsored_bills_avg,co_sponsored_bills_avg,joint_sponsored_bills_avg
100,Luke Clippinger,D,HD046,Rep,11447,0,27,0,7,20,...,15,0,4,11,0,0.555556,0.0,0.571429,0.550000,0.0
101,Kathleen Dumais,D,HD015,Rep,4580,0,96,0,27,69,...,37,0,6,31,0,0.385417,0.0,0.222222,0.449275,0.0
102,Tawanna Gaines,D,HD022,Rep,4660,0,54,0,1,53,...,26,0,0,26,0,0.481481,0.0,0.000000,0.490566,0.0
103,Maggie McIntosh,D,HD043,Rep,4649,0,74,0,5,69,...,35,0,5,30,0,0.472973,0.0,1.000000,0.434783,0.0
104,Pamela Queen,D,HD014,Rep,17917,0,78,0,9,69,...,29,0,2,27,0,0.371795,0.0,0.222222,0.391304,0.0
105,Jared Solomon,D,HD018,Rep,20796,0,128,0,6,122,...,52,0,2,50,0,0.406250,0.0,0.333333,0.409836,0.0
106,Lorig Charkoudian,D,HD020,Rep,20858,0,129,0,10,119,...,47,0,6,41,0,0.364341,0.0,0.600000,0.344538,0.0
107,Jheanelle Wilkins,D,HD020,Rep,19199,0,166,0,15,151,...,59,0,5,54,0,0.355422,0.0,0.333333,0.357616,0.0
108,Charlotte Crutchfield,D,HD019,Rep,20867,0,84,0,6,78,...,38,0,3,35,0,0.452381,0.0,0.500000,0.448718,0.0
109,Wanika Fisher,D,HD047B,Rep,20863,0,117,0,6,111,...,43,0,2,41,0,0.367521,0.0,0.333333,0.369369,0.0


In [59]:
# Output data
legislators.to_csv("legislators_2019.csv")

### Part 2: Find relationships between legislators ###

In [ ]:
# here we count how many times legislators have co sponsered each other

relationships = []
first_names = []
second_names = []
co_bills = []
for index,row in legislators.iterrows():
    name1 = row["name"]
    for index2,row2 in legislators.iterrows():
        name2 = row2["name"]
        names = [name1,name2]
        names = sorted(names)
        combo_name = names[0] + names[1]
        if (combo_name not in relationships):
            relationships.append(combo_name)
            first_names.append(names[0])
            second_names.append(names[1])
            co_bills = []
        
matrix = pd.DataFrame({
    "relationship": relationships,
    "first_name": first_names,
    "second_name" : second_names
})

In [ ]:
co_bills = []
co_bills_status = []
co_bills_total = []
print(len(matrix))
for index, row in matrix.iterrows():
    print(index)
    r_co_bills = []
    r_co_bills_status = []
    name1 = row["first_name"]
    name2 = row["second_name"]
    name1 = name1.strip()
    name2 = name2.strip()
    for index2, row2 in df.iterrows():
        name_one_found = False
        name_two_found = False
        primary_found = False
        secondary_found = False
        for sponsor_name, sponsor_type_id in zip(row2["sponsor_name"],row2["sponsor_type_id"]):
            sponsor_name = sponsor_name.strip()
            sponsor_type_id = sponsor_type_id.strip()
            if (sponsor_name == name1):
                name_one_found = True
                if (sponsor_type_id == "1"):
                    primary_found = True 
                elif (sponsor_type_id == "2"):
                    secondary_found = True
            elif (sponsor_name == name2):
                name_two_found = True
                if (sponsor_type_id == "1"):
                    primary_found = True 
                elif (sponsor_type_id == "2"):
                    secondary_found = True
        if (name_one_found and name_two_found and primary_found and secondary_found):
            print("_______FOUND__________")
            r_co_bills.append(row2["bill_id"])
            r_co_bills_status.append(row2["status"])
        
    co_bills.append(r_co_bills)
    co_bills_status.append(r_co_bills_status)
    co_bills_total.append(len(r_co_bills))        

matrix["co_bills"] = co_bills
matrix["co_bills_status"] = co_bills_status
matrix["co_bills_total"] = co_bills_total

In [ ]:
# output data

matrix = matrix.sort_values(by=['co_bills_total'], ascending=False)

matrix.to_csv("matrix_2019.csv")